<a href="https://colab.research.google.com/github/playeredlc/treinamento-h2ia/blob/master/Meta-heuristicas/busca_tabu_mochila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Metaheuristics
# Solving the Knapsack problem using Tabu Search

In [2]:
import numpy as np

In [3]:
class Knapsack:
  def __init__(self, weights, values, weight_limit):
    self.weights = weights
    self.values = values
    self.weight_limit = weight_limit
    
    self.items_amount = len(self.weights)
    self.state = self.generate_random_state()
  
  def set_state(self, state):
    self.state = state
  
  def get_state(self):
    return self.state

  def check_capacity(self, state=[]):
    if(len(state) == 0):
      state = self.state
    total_weight = 0
    
    for i in range(self.items_amount):
      if(state[i] != 0):
        total_weight += self.weights[i]

    return total_weight
  
  def check_value(self, state=[]):
    if(len(state) == 0):
      state = self.state

    total_value = 0
    for i in range(self.items_amount):
      if(state[i] != 0):
        total_value += self.values[i]   

    return total_value 

  def generate_random_state(self):
    state = np.zeros_like(self.values)
    partial_weight = 0

    while(partial_weight <= self.weight_limit):
      random_i = np.random.randint(0, self.items_amount)
      partial_weight += self.weights[random_i]

      if((partial_weight > self.weight_limit)):
        return state
      else:
        state[random_i] = 1
    
    return state

  def generate_neighbor_states(self):
    states_array = []

    for i in range(self.items_amount):
      new_state = np.copy(self.state)
      if(new_state[i] == 0):
        new_state[i] = 1
      else:
        new_state[i] = 0
      
      states_array.append(new_state)

    return states_array  


In [4]:
def get_best_state(knapsack, neighbor_states, tabu_list):

  best_index = -1
  best_value = -1
  # set first valid (not overweighted and not tabu) state as the best
  for i in range(len(neighbor_states)):
    state_weight = knapsack.check_capacity(neighbor_states[i])
    if(state_weight <= knapsack.weight_limit and i not in tabu_list):
      best_value = knapsack.check_value(neighbor_states[i])
      best_index = i
      break

  # look for better states
  for i in range(best_index, len(neighbor_states)):
    state_weight = knapsack.check_capacity(neighbor_states[i])
    state_value = knapsack.check_value(neighbor_states[i])

    if(state_weight <= knapsack.weight_limit):
      if(i not in tabu_list and state_value > best_value):
        best_value = knapsack.check_value(neighbor_states[i])
        best_index = i
  
  return best_value, best_index


In [5]:
def tabu_search(knapsack):
  iter = 0
  max_iter = 1000
  last_improvement = 0

  list_size = 100
  tabu_list = []
  
  best_ever = knapsack.get_state()

  while((iter - last_improvement) < max_iter):
    # generate states and evaluate the best
    neighbor_states = knapsack.generate_neighbor_states()
    best_value, best_index = get_best_state(knapsack, neighbor_states, tabu_list)

    # check if value improved
    if(knapsack.check_value(neighbor_states[best_index]) > knapsack.check_value(best_ever)):
      best_ever = neighbor_states[best_index]
      last_improvement = iter
      print(f'Best solution ever updated: {knapsack.check_value(best_ever)}')
    
    # update current state
    knapsack.set_state(neighbor_states[best_index])
    
    # update tabu list
    if(len(tabu_list) >= list_size):
      tabu_list.append(best_index)
      tabu_list.pop(0)
    else:
      tabu_list.append(best_index)
    
    iter += 1
  
  knapsack.set_state(best_ever)
  return knapsack


In [6]:
# knapsack definition
weights = np.array([63, 21, 2, 32, 13, 80, 19, 37, 56, 41, 14, 8, 32, 42, 7])
values = np.array([13, 2, 20, 10, 7, 14, 7, 2, 2, 4, 16, 17, 17, 3, 21])
weight_limit = 275
# create knapsack object with random state
knapsack = Knapsack(weights, values, weight_limit)

print(f'Initial Solution: {knapsack.get_state()}')
print(f'Initial Value: {knapsack.check_value()}')
print(f'Initial Weight: {knapsack.check_capacity()}\n')

solved_knapsack = tabu_search(knapsack)

print(f'\nSolution: {solved_knapsack.get_state()}')
print(f'Solution Value: {solved_knapsack.check_value()}')
print(f'Solution Weight: {solved_knapsack.check_capacity()}')


Initial Solution: [0 0 0 1 0 0 0 1 1 0 1 0 0 0 0]
Initial Value: 30
Initial Weight: 139

Best solution ever updated: 51
Best solution ever updated: 71
Best solution ever updated: 88
Best solution ever updated: 105
Best solution ever updated: 119
Best solution ever updated: 124
Best solution ever updated: 131
Best solution ever updated: 142

Solution: [1 0 1 1 1 1 1 0 0 0 1 1 1 0 1]
Solution Value: 142
Solution Weight: 270
